# Galaxy Simulation

## Kreisbahn der Erde um die Sonne 2D

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def calc_acceleration(force, mass):
    if mass <= 0:
        raise TypeError('Mass has to be greater than 0')
    return (1/mass)*force

def next_location(mass, position, speed, acceleration, delta_t):
    return (position + delta_t * speed + (delta_t**2/2)*acceleration)

def calc_gravitational_force(mass1, mass2, pos1, pos2):
    delta_pos = np.linalg.norm(pos2-pos1)
    return (6.673*10**-11)*(((mass1*mass2)/delta_pos**3)*(pos2-pos1))

def calc_mass_focus(ignore, masses, positions):
    total_mass = np.sum(masses)
    
    tmp_loc = np.zeros(3, dtype=np.float64)
    
    for i in range(masses.size):
        if i == ignore:
            continue
        tmp_loc = tmp_loc + (masses[i] * positions[i])    
    return (1/(total_mass - masses[i]))*tmp_loc



body_amount = 2
positions = np.zeros((body_amount, 3), dtype=np.float64)
speed = np.zeros((body_amount, 3), dtype=np.float64)
radius = np.zeros((body_amount), dtype=np.float64)
mass = np.zeros((body_amount), dtype=np.float64)

positions[0] = np.array([0,0,0])
speed[0] = [0,0,0]
mass[0] = 1.989*10**30

positions[1] = np.array([1.496*10**11, 0, 0])
speed[1] = np.array([0, 29780, 0])
mass[1] = 5.972*10**24

x_vals = []
y_vals = []

timestep = 100
step_amount = 315400

for i in range(step_amount):
    body = 1
    mass_foc_pos = positions[0]
    mass_foc_weight = mass[0]
    grav_force = calc_gravitational_force(mass[body], mass_foc_weight, positions[body], mass_foc_pos)
    accel = calc_acceleration(grav_force, mass[body])
    speed[body] = speed[body] + accel*timestep
    positions[body] = next_location(mass[body], positions[body], speed[body], accel, timestep)
    x_vals.append(positions[body][0])
    y_vals.append(positions[body][1])

plt.axis('equal')
plt.plot(x_vals, y_vals)

## Testing Cython

In [ ]:
%load_ext cython

### Version using NumPy Arrays only

In [ ]:
from numpy.linalg import norm
from numpy import array

G_CONSTANT = 6.673e-11

def g_force(mass1, mass2, pos1, pos2):
    
    delta_pos = pos2 - pos1
    abs_dpos = norm(delta_pos)
    return G_CONSTANT * (((mass1)/abs_dpos**3)*mass2) * delta_pos

### Cython optimised Version

In [ ]:
%%cython -a

cimport cython

import numpy as np
cimport numpy as np
from libc.math cimport sqrt

ctypedef np.float64_t DTYPE_t
cdef float G_CONSTANT_c = 6.673e-11

@cython.cdivision(True)
cpdef np.ndarray[DTYPE_t, ndim=1] g_force_cython(float mass1,
                                                 float mass2,
                                                 np.ndarray[DTYPE_t, ndim=1] pos1,
                                                 np.ndarray[DTYPE_t, ndim=1] pos2):
    
    cdef np.ndarray[DTYPE_t, ndim=1] d_pos = pos2 - pos1
    cdef float abs_dpos = sqrt(d_pos[0]**2+d_pos[1]**2+d_pos[2]**2)
    return G_CONSTANT_c * (((mass1)/abs_dpos**3)*mass2) * d_pos

In [ ]:
## Initialize masses and positions
import numpy as np

mass1 = 5.972 * 10**24
mass2 = 1.898 * 10**27
pos1 = np.array([108836377761.77997, 70586571455.50005, -1669580584.6400046])
pos2 = np.array([137140635336.02217, -37048853897.46875, 15369312079.6235])

#Vergleich:

%timeit g_force(mass1, mass2, pos1, pos2)
%timeit g_force_cython(mass1, mass2, pos1, pos2)

## Cython: *physics_formula*

In [ ]:
%load_ext Cython

### calc_acceleration()

In [ ]:
def calc_acceleration(force, mass):
    return force/mass

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np
cimport cython

ctypedef np.float64_t DTYPE_T

cpdef np.ndarray[DTYPE_T, ndim=1] calc_acceleration_cy(np.ndarray[DTYPE_T, ndim=1] force, float mass):
    return force/mass

In [ ]:
sun_weight = 1.989e30
earth_weight = 5.972e24
sun_pos = np.array([0, 0, 0], dtype=np.float64)
earth_pos = np.array([1.496e11, 0, 0], dtype=np.float64)
earth_force = g_force(earth_weight, sun_weight, earth_pos, sun_pos)

%timeit calc_acceleration(earth_force, earth_weight)
%timeit calc_acceleration_cy(earth_force, earth_weight)

### calc_mass_focus()

In [ ]:
def calc_mass_focus(masses, positions):
    tmp_focus = np.zeros(3, dtype=np.float64)
    for i in range(masses.size):
        tmp_focus = tmp_focus + masses[i] * positions[i]
    return tmp_focus/np.sum(masses)

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np
cimport cython

ctypedef np.float64_t DTYPE_T
        
@cython.cdivision(True)
@cython.boundscheck(False)
cpdef np.ndarray[DTYPE_T, ndim=1] calc_mass_focus_cy(np.ndarray[DTYPE_T, ndim=1] masses,
                                                     np.ndarray[DTYPE_T, ndim=2] positions):
    
    cdef np.ndarray[DTYPE_T, ndim=1] tmp_focus = np.empty(3, dtype=np.float64)
    cdef int i
    
    for i in range(masses.shape[0]):
        tmp_focus = tmp_focus + masses[i] * positions[i]
    return tmp_focus/np.sum(masses)

In [ ]:
from random import uniform 

positions = np.zeros((5000, 3), dtype=np.float64)
mass = np.zeros((5000), dtype=np.float64)

for i in range(mass.shape[0]):
    x_pos = uniform(1e10, 1e13)
    y_pos = uniform(1e10, 1e13)
    z_pos = uniform(1e10, 1e13)
    positions[i] = np.array([x_pos,
                             y_pos,
                             z_pos])

    mass[i] = uniform(1e25, 2e30)


%timeit calc_mass_focus(mass, positions)
%timeit calc_mass_focus_cy(mass, positions)

### calc_mass_focus_ignore()

In [ ]:
def calc_mass_focus_ignore(ignore, masses, positions):
    
    tmp_loc = np.zeros(3, dtype=np.float64)

    for i in range(masses.size):
        if i == ignore:
            continue
        tmp_loc = tmp_loc + (masses[i] * positions[i])
    return tmp_loc/(np.sum(masses) - masses[ignore])

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np
cimport cython

ctypedef np.float64_t DTYPE_T

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef np.ndarray[DTYPE_T, ndim=1] calc_mass_focus_ignore_cy(int ignore,
                                                            np.ndarray[DTYPE_T, ndim=1] masses,
                                                            np.ndarray[DTYPE_T, ndim=2] positions):
    
    cdef np.ndarray[DTYPE_T, ndim=1] tmp_loc = np.empty(3, dtype=np.float64)
    cdef int i
    for i in range(masses.shape[0]):
        if i == ignore:
            continue
        tmp_loc = tmp_loc + masses[i] * positions[i]
    return tmp_loc/(np.sum(masses) - masses[ignore])

In [ ]:
from random import uniform 

positions = np.zeros((5000, 3), dtype=np.float64)
mass = np.zeros((5000), dtype=np.float64)
ignore = 255

for i in range(mass.shape[0]):
    x_pos = uniform(1e10, 1e13)
    y_pos = uniform(1e10, 1e13)
    z_pos = uniform(1e10, 1e13)
    positions[i] = np.array([x_pos,
                             y_pos,
                             z_pos])

    mass[i] = uniform(1e25, 2e30)


%timeit calc_mass_focus_ignore(ignore, mass, positions)
%timeit calc_mass_focus_ignore_cy(ignore, mass, positions)

### calc_absolute_speed()

In [ ]:
def calc_absolute_speed(body_index, masses, positions):
    
    my_mass, my_position = masses[body_index], positions[body_index]
    mass_focus_ignored = calc_mass_focus_ignore(body_index, masses, positions)
    total_mass = np.sum(masses)

    abs_r = np.linalg.norm(my_position - mass_focus_ignored)
    return ((total_mass - my_mass) /
            total_mass)*np.sqrt(G_CONSTANT*total_mass/abs_r)

In [ ]:
%%cython
cimport cython
from libc.math cimport sqrt
import numpy as np
cimport numpy as np

ctypedef np.float64_t DTYPE_T

cdef float G_CONSTANT = 6.673e-11

# HAD TO COPY THIS METHOD IN, CAN BE IGNORED
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef np.ndarray[DTYPE_T, ndim=1] calc_mass_focus_ignore_cy(int ignore,
                                                            np.ndarray[DTYPE_T, ndim=1] masses,
                                                            np.ndarray[DTYPE_T, ndim=2] positions):
    cdef np.ndarray[DTYPE_T, ndim=1] tmp_loc = np.empty(3, dtype=np.float64)
    cdef int i
    for i in range(masses.shape[0]):
        if i == ignore:
            continue
        tmp_loc = tmp_loc + masses[i] * positions[i]
    return tmp_loc/(np.sum(masses) - masses[ignore])


cpdef float calc_absolute_speed_cy(int body_index,
                                   np.ndarray[DTYPE_T, ndim=1] masses,
                                   np.ndarray[DTYPE_T, ndim=2] positions):
    
    cdef float my_mass = masses[body_index]
    cdef np.ndarray[DTYPE_T, ndim=1] my_pos = positions[body_index]
    cdef float total_mass = np.sum(masses)
    
    cdef np.ndarray[DTYPE_T, ndim=1] to_norm = my_pos - calc_mass_focus_ignore_cy(body_index, masses, positions)
    cdef float abs_r = sqrt(to_norm[0]**2 + to_norm[1]**2 + to_norm[2]**2)
    return ((total_mass - my_mass) / total_mass) *sqrt(G_CONSTANT * total_mass/abs_r)

In [ ]:
from random import uniform 

positions = np.zeros((5000, 3), dtype=np.float64)
mass = np.zeros((5000), dtype=np.float64)
ignore = 255

for i in range(mass.shape[0]):
    x_pos = uniform(1e10, 1e13)
    y_pos = uniform(1e10, 1e13)
    z_pos = uniform(1e10, 1e13)
    positions[i] = np.array([x_pos,
                             y_pos,
                             z_pos])

    mass[i] = uniform(1e25, 2e30)


%timeit calc_absolute_speed(ignore, mass, positions)
%timeit calc_absolute_speed_cy(ignore, mass, positions)

# print(calc_absolute_speed(ignore, mass, positions))
# print(calc_absolute_speed_cy(ignore, mass, positions))